In [45]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.figure_factory as ff 


In [46]:
data = pd.read_csv('dataMining\dstest.csv')

In [47]:
data.head()

,company,live,mediaEngine,p2p,cdn,upload,peers_count,peers_match,timestamp,sessionDuration,playbackErrorCount,totalPlaybackErrorCount,content
0,Streamroot TV,VOD,ME1,0.0,8507376.0,0.0,0.0,1,1562469807407,120000.0,0,0,content-09235
1,Streamroot TV,VOD,ME1,0.0,5778744.0,0.0,0.0,0,1562469927310,240000.0,0,0,content-09235
2,Streamroot TV,VOD,ME1,0.0,7692960.0,0.0,0.0,0,1562470047330,360000.0,0,0,content-09235
3,Streamroot TV,VOD,ME1,0.0,8229888.0,0.0,0.0,0,1562470167720,480000.0,0,0,content-09235
4,Streamroot TV,VOD,ME1,0.0,10387000.0,0.0,0.0,0,1562470287353,600000.0,0,0,content-09235


In [48]:
## Question 1
## I would define the performace as the ratio of p2p over cdn for each company
## Having a high ratio means that most of the stream is supported by the peers and that is better

def ratio_p2p_cdn (df):
    return (df.p2p.mean()/df.cdn.mean()) * 100

data.groupby('company').apply(lambda df: ratio_p2p_cdn(df))

company
Flash Infos       65.031872
Sports News       28.190941
Streamroot TV    120.891015
dtype: float64

In [49]:
### We can also use this for checking the live streaming

data.groupby(['company', 'live']).apply(lambda df: ratio_p2p_cdn(df))

company        live
Flash Infos    Live    102.378527
               VOD      45.987885
Sports News    Live     31.544646
               VOD       1.951055
Streamroot TV  VOD     120.891015
dtype: float64

In [50]:
len(data)

385165

In [51]:
## Question 2.1

## We want those with the smallest sessionDuration, the first two minutes

data[data.sessionDuration == data.sessionDuration.min()]

,company,live,mediaEngine,p2p,cdn,upload,peers_count,peers_match,timestamp,sessionDuration,playbackErrorCount,totalPlaybackErrorCount,content
0,Streamroot TV,VOD,ME1,0.0,8507376.0,0.0,0.00,1,1562469807407,120000.0,0,0,content-09235
16,Streamroot TV,VOD,ME1,0.0,17608456.0,0.0,0.00,1,1562477387430,120000.0,0,0,content-17013
21,Streamroot TV,VOD,ME1,166409328.0,11303312.0,0.0,19.11,38,1562468705805,120000.0,0,0,content-08963
53,Streamroot TV,VOD,ME1,0.0,14936600.0,0.0,0.00,0,1562469688171,120000.0,0,0,content-14278
74,Streamroot TV,VOD,ME1,0.0,15740112.0,0.0,0.00,0,1562478964956,120000.0,0,0,content-16536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385101,Streamroot TV,VOD,ME1,122891464.0,1946928.0,0.0,6.22,9,1562478081892,120000.0,0,0,content-01158
385107,Streamroot TV,VOD,ME1,0.0,6409296.0,0.0,0.00,0,1562465908584,120000.0,0,0,content-04063
385124,Streamroot TV,VOD,ME1,0.0,3240932.0,0.0,0.00,1,1562466022339,120000.0,0,0,content-07336
385134,Streamroot TV,VOD,ME1,0.0,19570048.0,0.0,0.00,0,1562459158348,120000.0,0,0,content-15027


In [52]:
### Question 2.2

data = data.assign(isFirstPayload = data.sessionDuration == data.sessionDuration.min())

In [53]:
### Question 2.3

data[data.isFirstPayload].groupby('company').apply(len)

company
Flash Infos        805
Sports News        115
Streamroot TV    19914
dtype: int64

In [54]:
### Question 3.1
## For the 5 sessions:
### The playbackErrorCount will be 0, 1, 0, 1, 0
### The totalPlaybackErrorCount will be 0, 1, 1, 2, 2 

In [55]:
### Question 3.2

## Let's first identify the sessions and put them an id. We do not have extra information
## so we consider each session as distinct user
data['userID'] = 0

## Giving the distinct users an id
data.loc[data.isFirstPayload, 'userID']= data.loc[data.isFirstPayload].reset_index().index+1

## Here I find the userId for the rest of the sessions. 

data['row_index'] = data.index
int_sessions = data[data.isFirstPayload][['userID', 'content', 'timestamp', 'row_index']]
cont_sessions = data[data.isFirstPayload==False][['userID', 'content', 'timestamp', 'row_index']]

##the user has been watching the same content in 1 session
sessions_merge = pd.merge(int_sessions, cont_sessions, how='right', on='content')

## The difference in timestamp had to have a modulo 0 for intervals of 2 minutes
sessions_merge['diff_t'] = round(((sessions_merge.timestamp_y-sessions_merge.timestamp_x)/1000/60))%2

sessions_merge = sessions_merge[sessions_merge['diff_t'] == 0]
sessions_merge.userID_y = sessions_merge.userID_x

## Now let's add the userid to our initial data
data = pd.merge(data, sessions_merge, how='left', left_on='row_index', right_on='row_index_y')
data['userID'] = np.where(~data.isFirstPayload, data['userID_y'],  data['userID'])
data=data.iloc[:,:16]
data.rename(columns = {'content_x':'content'}, inplace=True) 



In [56]:
### Now let's find the sessions with at least one playback error

data.groupby('userID') \
            .apply(lambda df: df[df.totalPlaybackErrorCount>=1] \
                   .sort_values(['totalPlaybackErrorCount', 'sessionDuration']).tail(1))

,,company,live,mediaEngine,p2p,cdn,upload,peers_count,peers_match,timestamp,sessionDuration,playbackErrorCount,totalPlaybackErrorCount,content,isFirstPayload,userID,row_index
userID,,,,,,,,,,,,,,,,,
18.0,1021258,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,18.0,384449
106.0,1021259,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,106.0,384449
112.0,1021260,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,112.0,384449
139.0,1021261,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,139.0,384449
339.0,1021262,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,339.0,384449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20734.0,1021381,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,20734.0,384449
20739.0,1021382,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,20739.0,384449
20792.0,1021383,Flash Infos,Live,ME1,8820228.0,5038192.0,604300.0,7.62,40,1562479117964,1800000.0,0,3,content-15468,False,20792.0,384449


In [57]:
### Question 4.1
data['readableDate'] = data['timestamp'].apply(lambda x: (datetime.fromtimestamp(x/1000) - timedelta(minutes = 2)).strftime('%Y-%m-%d %H:%M:%S'))


In [98]:
### Question 4.2

selected_cont = data[data.content == 'content-05335']

def return_session (df):
    return df.readableDate.min(), df.readableDate.max()

concurrency= pd.DataFrame(selected_cont.groupby('userID').apply(return_session), columns = ['Duration']) 
chart_data = pd.DataFrame(concurrency['Duration'].tolist(), index=concurrency.index,columns = ['Start', 'Finish'])
   

In [99]:
chart_data.reset_index(inplace = True,level = 0)
chart_data.rename(columns={"userID": "Task"},inplace=True)


In [100]:
chart_data = chart_data.T.to_dict()
list_data = list(chart_data.values())

In [101]:
fig = ff.create_gantt(list_data) 
fig.show()

In [102]:
### Question 4.3

def concurrency(df):
    return df.assign(concurrency=df.shape[0])
    
selected_cont.groupby(['content', 'readableDate'])\
             .apply(concurrency).reset_index(drop=True) \
             .pipe(lambda df: df[df.concurrency<=10]) \
             .pipe(lambda df: ratio_p2p_cdn(df))

1910.8300997474596